## Game Recommendation System Using Steam API
1. App detail data cleaning, 
2. to understand the details of games from steam api
3. build databased with Postgresql
4. four models on recommending games

In [1]:
import requests, json, os, sys, time, re
from bs4 import BeautifulSoup
from datetime import datetime
from sqlalchemy import *
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity
from sklearn.cluster import KMeans
import numpy as np
from pyspark.mllib.recommendation import ALS
from pyspark import SparkContext

## Data Cleaning on the Obtained Game App Detail Data, turn the data into a clean dataframe
### only 1000 examples used.

In [13]:
with open('data/app_detail.txt', 'r+') as file:
    dic_steam_app = {'app_price':{},'name':{},'score':{},'success':{}, 'windows':{},'mac':{},\
                     'linux':{},'type':{},'release_date':{},'recommendation':{},'header_image':{}}
    list_of_strings = file.readlines()
    dic_about_the_game = {}
    for raw_string in list_of_strings[:1000]:
        app_data = list(json.loads(raw_string).values())[0]
        if app_data == {}:
            dic_steam_app['success'].update({steam_id:False})
        if app_data != {}:
            steam_id = app_data.get('steam_appid')
            app_name = app_data.get('name')
            header_image = app_data.get('header_image')
            app_type = app_data.get('type')
            critic_score = app_data.get('metacritic', {}).get('score')
            app_price = app_data.get("price_overview", {}).get('initial')
            if app_data.get('is_free') != False:
                app_price = 0
                

            for (platform, is_supported) in app_data.get('platforms',{}).items():
                if is_supported == True:
                    dic_steam_app[platform].update({steam_id:1})
                else:
                    dic_steam_app[platform].update({steam_id:0})
            
            release_date = app_data.get("release_date",{}).get('date')
            if app_data.get('release_date',{}).get('coming_soon') == False:
                about_the_game = app_data.get('about_the_game')
                soup = BeautifulSoup(about_the_game,'lxml')
                game_description = re.sub(r'(\s+)',' ',soup.text).strip()
                dic_about_the_game.update({steam_id:game_description})
                release_date = app_data.get('release_date',{}).get('date')
            
                if not release_date == '':
                    if re.search(',', release_date) == None:
                        release_date = datetime.strptime(release_date, '%b %Y')
                    else:
                        try:
                            release_date = datetime.strptime(release_date, '%b %d, %Y')
                        except:
                            release_date = datetime.strptime(release_date, '%d %b, %Y')       # "release_date": {"date": "Sep 23, 2014", 
            recommendation = app_data.get('recommendations',{}).get('total')
            dic_steam_app['app_price'].update({steam_id:app_price})
            dic_steam_app['name'].update({steam_id:app_name})
            dic_steam_app['score'].update({steam_id:critic_score})
            dic_steam_app['type'].update({steam_id:app_type})
            dic_steam_app['release_date'].update({steam_id:release_date})
            dic_steam_app['recommendation'].update({steam_id:recommendation})
            dic_steam_app['header_image'].update({steam_id:header_image})    



def converter_time(date):
    return datetime.strptime(date, '%b %d, %Y')

In [14]:
df_steam_app = pd.DataFrame(dic_steam_app)
df_steam_app.app_price = df_steam_app.app_price.map(lambda x: x/100.0)
df_steam_app.index.name = 'steam_appid'
df_steam_app['windows'] = df_steam_app.windows.fillna(0)
df_steam_app['mac'] = df_steam_app.mac.fillna(0)
df_steam_app['linux'] = df_steam_app.linux.fillna(0)
df_steam_app = df_steam_app[['name', 'type', 'app_price', "success", 'release_date', 'score', 'recommendation', 'windows', 'mac', 'linux', 'success','header_image']]
df_steam_app.reset_index(inplace=True)
df_steam_app.success.fillna(True, inplace=True)

df_steam_app.to_csv('Cleaned_app_data.csv')

In [15]:
df_steam_app.shape

(998, 13)

## Solving the Encoding Problem with unicode and utf-8 for the Column - 'name'

In [16]:
with open('data/2017-04-14.json','rb') as f:
    dic_steamspy = json.load(f)
df_new = pd.read_csv("Cleaned_app_data.csv", encoding='utf-8')
df_new['name'] = df_new['name'].map(lambda x: x.encode('unicode-escape').decode('utf-8'))

In [17]:
df_new.head()

,Unnamed: 0,steam_appid,name,type,app_price,success,release_date,score,recommendation,windows,mac,linux,success.1,header_image
0,0,300,Day of Defeat: Source,game,9.99,NaN,2010-07-12,80.0,7753.0,1,1,1,NaN,http://cdn.akamai.steamstatic.com/steam/apps/3...
1,1,1700,Arx Fatalis,game,4.99,NaN,2007-04-03,77.0,456.0,1,0,0,NaN,http://cdn.akamai.steamstatic.com/steam/apps/1...
2,2,2300,DOOM II,game,4.99,NaN,2007-08-03,83.0,1553.0,1,0,0,NaN,http://cdn.akamai.steamstatic.com/steam/apps/2...
3,3,2700,RollerCoaster Tycoon\xae 3: Platinum,game,19.99,NaN,2008-03-12,81.0,2773.0,1,1,0,NaN,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,4,2720,ThreadSpace: Hyperbol,game,9.99,NaN,2007-07-12,NaN,NaN,1,0,0,NaN,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [18]:
import pandas as pd
from sqlalchemy import create_engine

# create engine to postgressql 
engine = create_engine(u"postgresql://postgres@localhost:5432/Game Recommendation", encoding='utf-8')

# write file to sql
df_new.to_sql('tbl_app_info_test', engine, if_exists='replace')

## try a query
df_sql = pd.read_sql("SELECT count(linux) FROM tbl_app_info_test", con=engine)    

# chunks = pd.read_csv('Cleaned_app_data.csv', chunksize=10000)
# for chunk in chunks:
#     chunk.to_sql(name='table', if_exist='append', con=engine)

In [19]:
a = engine.execute(
    '''
    SELECT * FROM tbl_app_info_test LIMIT 5;
    ''')
a.fetchall()

[(0, 0, 300, 'Day of Defeat: Source', 'game', 9.99, None, '2010-07-12', 80.0, 7753.0, 1, 1, 1, None, 'http://cdn.akamai.steamstatic.com/steam/apps/300/header.jpg?t=1447350820'),
 (1, 1, 1700, 'Arx Fatalis', 'game', 4.99, None, '2007-04-03', 77.0, 456.0, 1, 0, 0, None, 'http://cdn.akamai.steamstatic.com/steam/apps/1700/header.jpg?t=1478190772'),
 (2, 2, 2300, 'DOOM II', 'game', 4.99, None, '2007-08-03', 83.0, 1553.0, 1, 0, 0, None, 'http://cdn.akamai.steamstatic.com/steam/apps/2300/header.jpg?t=1449848674'),
 (3, 3, 2700, 'RollerCoaster Tycoon\\xae 3: Platinum', 'game', 19.99, None, '2008-03-12', 81.0, 2773.0, 1, 1, 0, None, 'http://cdn.akamai.steamstatic.com/steam/apps/2700/header.jpg?t=1473276734'),
 (4, 4, 2720, 'ThreadSpace: Hyperbol', 'game', 9.99, None, '2007-07-12', None, None, 1, 0, 0, None, 'http://cdn.akamai.steamstatic.com/steam/apps/2720/header.jpg?t=1447351073')]

## Connect the PostgresSql database to read the tables

In [20]:
# import psycopg2 as pg
# import pandas.io.sql as psql

# connection = pg.connect("dbname=newdvddatabase user=postgres")

# dataframe = psql.read_sql("SELECT city_id FROM city", connection)


## Building Recommendation Model


In [21]:
df_steam_app.head()

,steam_appid,name,type,app_price,success,release_date,score,recommendation,windows,mac,linux,success,header_image
0,300,Day of Defeat: Source,game,9.99,NaN,2010-07-12,80.0,7753.0,1,1,1,NaN,http://cdn.akamai.steamstatic.com/steam/apps/3...
1,1700,Arx Fatalis,game,4.99,NaN,2007-04-03,77.0,456.0,1,0,0,NaN,http://cdn.akamai.steamstatic.com/steam/apps/1...
2,2300,DOOM II,game,4.99,NaN,2007-08-03,83.0,1553.0,1,0,0,NaN,http://cdn.akamai.steamstatic.com/steam/apps/2...
3,2700,RollerCoaster Tycoon® 3: Platinum,game,19.99,NaN,2008-03-12,81.0,2773.0,1,1,0,NaN,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,2720,ThreadSpace: Hyperbol,game,9.99,NaN,2007-07-12,NaN,NaN,1,0,0,NaN,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [22]:
df_steam_app = pd.read_sql('tbl_app_info_test',engine)

df_valid_games = df_steam_app.query('type == "game" and release_date <= "{}" and app_price >= 0'.format(datetime.today().date().isoformat()))
set_valid_game_id = set(df_valid_games.steam_appid)


### Popularity Based

In [23]:
df_popularity_based_results = pd.Series(list(dic_steamspy.get('owners').values()), \
                                        list(dic_steamspy.get('owners').keys())).sort_values(ascending=False).to_frame()
df_popularity_based_results.index.name = "steam_appid"

df_popularity_based_results.reset_index(inplace=True)
df_popularity_based_results.to_sql('tbl_results_popularity_based',engine,if_exists='replace')

In [24]:
df_popularity_based_results.head()

,steam_appid,0
0,570,99137784
1,440,39401354
2,304930,28766051
3,730,28139418
4,550,16036217


game ranked based on popularity

### Content Based - Description 

In [25]:
list_tem = list(set(dic_about_the_game.keys()) - set_valid_game_id)
for i in list_tem:
    del dic_about_the_game[i]

calculate tfidf on the game description

In [26]:
tfidf = TfidfVectorizer(strip_accents='unicode', stop_words='english').fit_transform(dic_about_the_game.values())

In [27]:
dic_recomended = {} # 996 numbers of games , 17347 contents
lst_app_id = list(dic_about_the_game.keys())

In [28]:
for index in range(tfidf.shape[0]):
    cosine_similarity = linear_kernel(tfidf[index:index+1], tfidf).flatten()
    related_docs_indices = cosine_similarity.argsort()[-2:-22:-1]
    dic_recomended.update({lst_app_id[index]:[lst_app_id[i] for i in related_docs_indices]})
    
df_content_based_results = pd.DataFrame(dic_recomended).T
df_content_based_results.index.name = 'steam_appid'
df_content_based_results.reset_index(inplace=True)
df_content_based_results.to_sql('tbl_results_content_based',engine,if_exists='replace')

In [29]:
df_content_based_results.head()

,steam_appid,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,300,34870,536630,55100,7940,49520,4520,41060,321800,393740,...,453810,361970,405820,454520,413030,235900,64000,364260,389690,12830
1,1700,238750,547810,277650,229520,503530,242840,310510,403510,553910,...,63900,502750,424170,463000,364260,424370,216910,418960,429250,437710
2,2300,39630,289300,3270,514050,357830,224900,22670,216930,254100,...,8880,461030,312240,310510,227900,219070,369180,324450,440630,487030
3,2700,493340,282560,378420,12580,402800,36150,239820,276380,411750,...,422270,332780,545960,545920,339860,315460,412440,412460,322900,362860
4,2720,311170,244850,330100,351300,255500,224900,218510,338330,321350,...,463920,10530,265590,107200,252450,49300,46790,495580,23500,414950


## Item Based Model

In [30]:
dic_purchase = {}

path_user_inventory = 'data/user_inventory_sample.txt'
with open(path_user_inventory, 'rb') as f:
    lst_all = f.readlines()
    for i in lst_all:
        user_id, user_inventory = list(json.loads(i).items())[0]
        #print("user_id", "user_inventory", user_id, user_inventory)
        if user_inventory != [] and user_inventory != None and user_inventory != {}:
            dic_purchase[user_id] = {}
            for play_info in user_inventory:
                appid = play_info.get('appid')
                #print(appid)
                if appid in set_valid_game_id:
                   # print("hell")
                    dic_purchase[user_id].update({appid:1})

#df_purchase = pd.DataFrame(columns = dic_purchase).fillna(0)


In [31]:
#pd.DataFrame(data=dic_purchase)

purchase_matrix = pd.DataFrame(dic_purchase).fillna(0)
purchase_matrix.values

array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [32]:
lst_user_id=purchase_matrix.columns
lst_app_id =purchase_matrix.index

In [33]:
lst_user_id

Index(['76561197960355015', '76561197960385706', '76561197960422789',
       '76561197960464402', '76561197960477514', '76561197960562426',
       '76561197960699459', '76561197960893141', '76561197961091730',
       '76561197961243455',
       ...
       '76561198258604825', '76561198258639801', '76561198258679897',
       '76561198258749191', '76561198259586242', '76561198259723921',
       '76561198259865770', '76561198259925540', '76561198259974871',
       '76561198260675813'],
      dtype='object', length=4460)

In [34]:
lst_app_id

Int64Index([   300,   1700,   2300,   2700,   2720,   3000,   3020,   3270,
              3510,   3530,
            ...
            464620, 467120, 467960, 473520, 473710, 477170, 485870, 486150,
            487220, 488210],
           dtype='int64', length=662)

In [35]:
dic_recomended_item_based = {}
for index in range(len(lst_app_id)):
    cosine_similarities = linear_kernel(purchase_matrix[index:index+1], purchase_matrix).flatten()
    lst_related_app = np.argsort(cosine_similarities)[-1:-101:-1]
    dic_recomended_item_based.update({lst_app_id[index]:[lst_app_id[i] for i in lst_related_app]})
    
df_item_based_result = pd.DataFrame(dic_recomended_item_based).T

In [36]:
df_item_based_result.index.name = 'steam_appid'
df_item_based_result.reset_index(inplace=True)
df_item_based_result.to_sql('tbl_results_item_based',engine,if_exists='replace')


In [37]:
df_item_based_result.head()

,steam_appid,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,300,300,72850,49520,271590,8190,22380,238460,10180,12220,...,243950,398710,9500,200940,207650,224920,70110,428690,201420,6810
1,1700,1700,49520,72850,271590,22380,238460,8190,220240,244850,...,312610,276810,268750,318530,322920,326410,207650,244730,58550,2700
2,2300,2300,49520,72850,8190,22380,238460,271590,12220,220240,...,226120,254440,33950,252630,13530,9500,268870,70110,383580,256290
3,2700,2700,72850,49520,22380,271590,8190,239820,238460,12220,...,38600,254440,218510,42170,236870,9500,231040,55100,70660,243950
4,2720,72850,22380,49520,2720,367500,252630,254440,247660,233290,...,12200,431290,6210,23380,2700,3020,2300,220200,283980,70640


## Collaborative Filtering
### 'ALS' Model using Pyspark

In [38]:
sc = SparkContext()

Helper Functions:

In [48]:
def parse_raw_string(raw_string):
    user_inventory = list(json.loads(raw_string).items())
    return user_inventory[0]

def id_index(x):
    ((user_id,lst_inventory),index) = x
    return (index, user_id)


def create_tuple(x):
    ((user_id,lst_inventory),index) = x
    if lst_inventory != None:
        return (index, [(i.get('appid'), 1) for i in lst_inventory if i.get('appid') in set_valid_game_id])
    else:
        return (index, [])


In [40]:
user_inventory_rdd = sc.textFile("data/user_inventory_sample.txt").map(parse_raw_string).zipWithIndex()
dic_id_index = user_inventory_rdd.map(id_index).collectAsMap()
training_rdd = user_inventory_rdd.map(create_tuple).flatMapValues(lambda x : x).map(lambda x:(x[0],x[1][0],x[1][1]))
model = ALS.train(training_rdd, 5)

In [41]:
dic_recommended = {}
for index in dic_id_index.keys():
    try:
        lst_recommended = [i.product for i in model.recommendProducts(index,50)]
        user_id = dic_id_index.get(index)
        dic_recommended.update({user_id:lst_recommended})
    except:
        pass

In [44]:
df_als_result = pd.DataFrame(dic_recommended).T
df_als_result.index.name = 'user_id'
df_als_result.reset_index(inplace=True)
df_als_result.to_sql('tbl_results_als_based',engine,if_exists='replace')

In [46]:
df_als_result.head()

,user_id,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,76561197960355015,388800,244690,279520,46560,16020,263940,360580,375520,276380,...,298830,16060,32120,376150,389200,367340,337730,339860,365400,365440
1,76561197960385706,388800,244690,46560,252170,397440,425160,315650,16020,395910,...,376780,365400,376150,16060,367340,32120,375580,389200,365440,337730
2,76561197960422789,388800,244690,46560,219070,368710,252170,391580,27800,337670,...,365260,346950,293440,234350,16000,327400,395570,276240,282860,378590
3,76561197960464402,388800,244690,46560,219070,397330,355100,276380,263940,27800,...,376150,365400,399900,367340,32120,16060,337730,365440,375580,389200
4,76561197960477514,388800,46560,219070,244690,397330,370270,395910,337670,385060,...,234350,16000,395570,333300,352370,357720,341020,358750,377940,384100
